In [1]:
import requests
import json

def fetch_valid_card_names():
    valid_card_names = set()
    url = "https://api.scryfall.com/catalog/card-names?q=legal:commander"
    while url:
        response = requests.get(url)
        data = json.loads(response.text)
        cards = data.get("data", [])

        for card in cards:
            valid_card_names.add(card)  # No need to access a "name" key

        if data.get("has_more"):
            url = data["next_page"]
        else:
            break

    return valid_card_names

valid_card_names = fetch_valid_card_names()


In [2]:
# Cell 1 - Split decks into main decks and sideboards
import os

data_dir = r'C:\Users\Devin\Desktop\MTG Deckbuilding Algorithm\data'
deck_files = os.listdir(data_dir)
main_decks = []
sideboards = []

for deck_file in deck_files:
    file_path = os.path.join(data_dir, deck_file)
    with open(file_path, 'r', encoding='utf-8', errors='replace') as file:
        deck_lines = file.readlines()
        sideboard_index = deck_lines.index('Sideboard\n')
        main_deck = ''.join(deck_lines[:sideboard_index])
        sideboard = ''.join(deck_lines[sideboard_index+1:])
        main_decks.append(main_deck)
        sideboards.append(sideboard)

# Cell 1 - Combine the sideboard (commander) and main deck
combined_decks = []

for main_deck, sideboard in zip(main_decks, sideboards):
    combined_decks.append(sideboard + main_deck)


In [3]:
# Cell 2 - Tokenize the combined decks
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(char_level=False, filters='', split='\n')
tokenizer.fit_on_texts(combined_decks)
total_words = len(tokenizer.word_index)

# Cell 2 - Convert text to sequences
sequences = tokenizer.texts_to_sequences(combined_decks)

# Cell 2 - Prepare input and target sequences
seq_length = 100
input_data = []
target_data = []

for deck_seq in sequences:
    for i in range(len(deck_seq) - seq_length):
        input_data.append(deck_seq[i:i + seq_length])
        target_data.append(deck_seq[i + seq_length])

input_data = np.array(input_data)
target_data = np.array(target_data)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

model = Sequential([
    Embedding(total_words + 1, 256, input_length=seq_length),
    LSTM(256, return_sequences=True),
    LSTM(256),
    Dense(total_words + 1, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')


In [5]:
model.fit(input_data, target_data, epochs=10, batch_size=16)

Epoch 1/10
2/2 [==============================] - 5s 307ms/step - loss: 8.0403
Epoch 2/10
2/2 [==============================] - 1s 323ms/step - loss: 7.9848
Epoch 3/10
2/2 [==============================] - 1s 293ms/step - loss: 7.6867
Epoch 4/10
2/2 [==============================] - 1s 307ms/step - loss: 6.0222
Epoch 5/10
2/2 [==============================] - 1s 326ms/step - loss: 4.7439
Epoch 6/10
2/2 [==============================] - 1s 304ms/step - loss: 3.5401
Epoch 7/10
2/2 [==============================] - 1s 303ms/step - loss: 2.5008
Epoch 8/10
2/2 [==============================] - 1s 307ms/step - loss: 1.8588
Epoch 9/10
2/2 [==============================] - 1s 296ms/step - loss: 1.5393
Epoch 10/10
2/2 [==============================] - 1s 303ms/step - loss: 1.3289


In [6]:
def generate_cards(commander, num_cards):
    # This function should be implemented using the trained model.
    # It should generate 'num_cards' cards based on the given 'commander'.
    pass

In [7]:
output_dir = r'C:\Users\Devin\Desktop\MTG Deckbuilding Algorithm\output'
output_file = os.path.join(output_dir, 'generated_deck.txt')

with open(output_file, 'w', encoding='utf-8') as file:
    file.write(generated_text)


NameError: name 'generated_text' is not defined